# This notebook contains code for replacing a pseudoatom (currently P) with another, constant substructure.  The following functions are currently configured to substitute a trivalent P atom with an alpha-amino backbone plus beta-carbon (e.g. an alanine moiety).

## Enter input and output filenames (must be in .sdf format) in cell below

In [ ]:
in_file = "C:\\Users\\umbci\\Desktop\\MAYGEN\\DEST_S-P.sdf"
out_file = "C:\\Users\\umbci\\Desktop\\MAYGEN\\Libraries\\DEST_S.sdf"

### If you encounter a SyntaxError mentioning a unicode error, make sure there are no single backslashes (\\) in the file names.  Change any single backslashes to double backslashes (\\\\) or forward slashes (/).

## Module imports below

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdChemReactions

## Function definitions below

In [ ]:
def pseudoReplace(mol):
    """Replace pseudoatom with alpha-amino acid backbone.
    
    Replaces trivalent phosphorous atom with an alpha-amino acid backbone and beta-carbon.
    SMARTS patterns can be modified to accommodate different pseudoatom symbols, valences, or replacement substructures

    Parameters
    ----------
    mol : RDKit Mol object
        Mol object with trivalent P pseudo atom

    Returns
    -------
    RDKit Mol object with P pseudo atom replaced.  Returns input Mol if P atom is not present.

    """
    # SMARTS patterns for different numbers of P heavy-atom neighbors
    three_neighbor = Chem.MolFromSmarts("[PD3,pD3]")
    two_neighbor = Chem.MolFromSmarts("[PD2,pD2]")
    one_neighbor = Chem.MolFromSmarts("[PD1,pD1]")
    three_h = Chem.MolFromSmarts("[PX3&H3]")
    # first match # of P non-H neighbors, then bonding patterns among those neighbors; return input molecule if no matches
    if mol.HasSubstructMatch(three_neighbor): # P with 0 hydrogens
        rxn = rdChemReactions.ReactionFromSmarts("[PD3,pD3]([*:2])([*:3])[*:1]>>NC(C(=O)O)C([*:2])([*:3])[*:1]")
        return rxn.RunReactant(mol,0)[0][0]
    elif mol.HasSubstructMatch(two_neighbor): # P with 2 neighbors
        if mol.HasSubstructMatch(Chem.MolFromSmarts("[PD2,pD2]=[*]")):
            rxn = rdChemReactions.ReactionFromSmarts("[PD2,pD2]([*:2])=[*:1]>>NC(C(=O)O)C([*:2])=[*:1]")
            return rxn.RunReactant(mol,0)[0][0]
        elif mol.HasSubstructMatch(Chem.MolFromSmarts("[PD2,pD2](:[*]):[*]")): # aromatic P
            if mol.HasSubstructMatch(Chem.MolFromSmarts("[pH]")): # aromatic pH
                rxn = rdChemReactions.ReactionFromSmarts("[PD2,pD2](:[*:2]):[*:1]>>NC(C(=O)O)C([*:2])[*:1]")
                return rxn.RunReactant(mol,0)[0][0]
            else: # aromatic P, beta-carbon needs to be aromatic too
                rxn = rdChemReactions.ReactionFromSmarts("[PD2,pD2](:[*:2]):[*:1]>>NC(C(=O)O)c(:[*:2]):[*:1]")
                return rxn.RunReactant(mol,0)[0][0]
        else: # P with 2 single bonded neighbors
            rxn = rdChemReactions.ReactionFromSmarts("[PD2,pD2]([*:2])[*:1]>>NC(C(=O)O)C([*:2])[*:1]")
            return rxn.RunReactant(mol,0)[0][0]
    elif mol.HasSubstructMatch(one_neighbor): # P with 1 neighbor
        if mol.HasSubstructMatch(Chem.MolFromSmarts("[PD1]#[*]")):
            rxn = rdChemReactions.ReactionFromSmarts("[PD1]#[*:1]>>NC(C(=O)O)C#[*:1]")
            return rxn.RunReactant(mol,0)[0][0]
        elif mol.HasSubstructMatch(Chem.MolFromSmarts("[PD1]=[*]")):
            rxn = rdChemReactions.ReactionFromSmarts("[PD1]=[*:1]>>NC(C(=O)O)C=[*:1]")
            return rxn.RunReactant(mol,0)[0][0]
        else:
            rxn = rdChemReactions.ReactionFromSmarts("[PD1][*:1]>>NC(C(=O)O)C[*:1]")
            return rxn.RunReactant(mol,0)[0][0]
    elif mol.HasSubstructMatch(three_h): # P with 3 hydrogens, replace with Ala
        return Chem.MolFromSmiles("NC(C(=O)O)C")
    else: # if P pseudo atom isn't present in input molecule
        return mol

## Main program

In [ ]:
# load in data file
data = Chem.SDMolSupplier(in_file, removeHs=True)
# create file writer
w = Chem.SDWriter(out_file)
for m in range(len(data)):
    #mol2 = Chem.MolFromSmiles(mol)
    print(m, end='\r', flush=True)
    new_mol = pseudoReplace(data[m])
    w.write(new_mol)
w.close()

### If you get an error about an index being out of range, it's likely because an input molecule has one of the pseudoatom substructures defined in the pseudoReplace() function, but the SMARTS string for the reaction can't match the environment around the pseudoatom.  These patterns should work for the use cases outlined in the JoVE paper; if there are any changes to the pseudoatom used or the substructure that would replace it, inspect the input molecule that caused the error and adjust the reaction rules to account for it.